In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/clean/opensky_asia_clean.csv")
df.head()

,timestamp,icao24,callsign,latitude,longitude,baro_altitude,velocity,vertical_rate,true_track,on_ground
0,2026-02-13 16:28:11,80162c,AXB321,22.4359,49.5145,10363.20,217.83,0.00,280.75,False
1,2026-02-13 16:28:11,801640,AIC2303,16.4021,96.0041,10965.18,233.14,0.33,303.94,False
2,2026-02-13 16:28:11,801645,AIC1863,28.5596,77.0890,NaN,0.00,NaN,278.44,True
3,2026-02-13 16:28:11,80162f,AIC212,28.6025,81.5784,10363.20,175.82,0.00,286.31,False
4,2026-02-13 16:28:11,80163c,IGO243H,25.5450,76.8816,11894.82,246.10,0.00,177.72,False


### Feature introduction:
1. timestamp: timestamp at which data was recorded.
2. icao24: Unique ICAO 24-bit address of the transponder in hex string representation.
3. callsign: 